# Exact Diagonalization of $J$-$K$ Hamiltonian
    Alan Morningstar
    May 2017

## Include Source

In [1]:
include("../src/HeisenbergED.jl");

## Main

#### specify parameters

In [2]:
# square lattice length
const Lx = 4
const Ly = 4
# number of sites
const N = Lx*Ly
# NN coupling
const J1 = 1.0
# NNN coupling
const J2 = 0.4
# plaquette coupling
const K = 0.3

# choose Sz sector
const Sz = 0
# choose kx,ky by specifying mi such that mi is in 0:Li-1
const mx = 0
const my = 0
const kx = 2*pi*mx/Lx
const ky = 2*pi*my/Ly
# choose spin inversion quantum number
const z = 1

# number of eigenvalues desired
const numEigs = 4
# a tolerance for error
const tolerance = 10.^(-8.)
# ritzVec = true if you want the eigenvectors returned too
const ritzVec = true
# number of Krylov vectors in eigenvalue calculation
const numKrylovVecs = 10
# maximum number of iterations to converge eigenvalues
const maxIter = 200

# plaquette (x,y) vectors, locating p1,p2,p3,p4 on the plaquette of the p1 site and p1D,p2D,p1L,p3L on adjacent plaquettes
const neighborVectors = [(0,0),(1,0),(0,1),(1,1),(0,-1),(1,-1),(-1,0),(-1,1)];

#### define lattice, symmetry sector, and basis

In [3]:
# define the lattice
const l = lattice(Lx,Ly,neighborVectors);

In [4]:
# specify symmetry sector
const s = sector(Sz,kx,ky,z);

In [5]:
# construct the basis
# 0.34 seconds for 6x4 lattice, basis is ~0.002 GB
@time const basis = reducedBasis{UInt64}(l,s)
println("Dimension of reduced Hilbert space is ",basis.dim,".");

  0.102415 seconds (164.93 k allocations: 7.569 MB, 5.67% gc time)
Dimension of reduced Hilbert space is 441.


In [6]:
(sizeof(basis.b)+sizeof(basis.n))/10^9

7.056e-6

#### build the Hamiltonian

In [10]:
# couplings type to make passing J1,K easier
const c = couplings(J1,J2,K)

# build the sparse Hamiltonian
@time const H = constructSparseHam(basis,c,s,l);

  0.582349 seconds (663.35 k allocations: 23.633 MB, 1.67% gc time)


In [11]:
println("H: ",(sizeof(H.colPntrs)+sizeof(H.rowIndcs)+sizeof(H.nzVals)+sizeof(H.nzPntrs))/10^9," GB of memory.")

H: 8.8584e-5 GB of memory.


#### find eigenvalues and eigenvectors

In [15]:
# compute eigenvalues
@time eigsResult = eigs(H; nev=numEigs,ncv=numKrylovVecs,maxiter=maxIter, which=:SR, tol=tolerance, ritzvec=ritzVec);

  1.695641 seconds (2.21 M allocations: 85.179 MB, 1.22% gc time)


In [19]:
# print energies
println("Energies are: ")
for en in real(eigsResult[1])
    println(en)
end
# print algorithm performance
println("Number of iterations = ",eigsResult[3])
println("Number of matrix-vector multiplications = ",eigsResult[4])

Energies are: 
-6.694896080039535
-6.735843696892488
-6.730790539793388
-6.2082326260702425
Number of iterations = 4
Number of matrix-vector multiplications = 35


#### compute $S^2$ values and save spectral data
Save in format |  E  |  S(S+1)  |  Sz  |  mx  |  my  |  z  |

In [21]:
using DataFrames

In [22]:
# energies
EData = real(eigsResult[1])
# Sz values
SzData = fill(Sz,numEigs)
# mx values
mxData = fill(mx,numEigs)
# my values
myData = fill(my,numEigs)
# z values
zData = fill(z,numEigs);

In [23]:
# S(S+1) values
@time S2Data = round(Int64,real(S2expectations(basis,s,l,eigsResult[2])));

  0.223021 seconds (154.56 k allocations: 5.733 MB)


In [25]:
# create DataFrame
df = DataFrame(E=EData,Ssqrd=S2Data,Sz=SzData,mx=mxData,my=myData,z=zData)

,E,Ssqrd,Sz,mx,my,z
1,-6.994896080039535,0,0,0,0,1
2,-6.735843696892488,0,0,0,0,1
3,-6.730790539793388,0,0,0,0,1
4,-6.2082326260702425,6,0,0,0,1
